Copyright 2023 Genentech, Inc.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
import sys
sys.path.append('/mnt/ps/home/CORP/safiye.celik/')
sys.path.append('/mnt/ps/home/CORP/safiye.celik/EFAAR_benchmarking')

from efaar_benchmarking.data_loading import load_replogle
from efaar_benchmarking.efaar import *
from efaar_benchmarking.benchmarking import benchmark
from efaar_benchmarking.plotting import plot_recall

recall_threshold_pairs = []
start = 0.01
end = 0.99
step = 0.01

while start <= .105 and end >= .895:
    recall_threshold_pairs.append((round(start,2), round(end,2)))
    start += step
    end -= step

print(recall_threshold_pairs)

## PCA Embeddings

In [ ]:
adata = load_replogle("genome_wide", "normalized")
metadata = adata.obs
print(metadata.shape)
embeddings = embed_by_pca_anndata(adata)
print(embeddings.shape)
del adata
embeddings = align_on_controls(embeddings, metadata)
print(embeddings.shape)
map_data = aggregate(embeddings, metadata)
del embeddings, metadata
metrics = benchmark(map_data, recall_thr_pairs=recall_threshold_pairs)
plot_recall(metrics)
metrics.groupby('source')['recall_0.05_0.95'].mean()

## scVI Embeddings

In [ ]:
adata = load_replogle("genome_wide", "raw")
metadata = adata.obs
print(metadata.shape)
embeddings = embed_by_scvi_anndata(adata)
print(embeddings.shape)
del adata
embeddings = align_on_controls(embeddings, metadata)
map_data = aggregate(embeddings, metadata)
del embeddings, metadata
metrics = benchmark(map_data, recall_thr_pairs=recall_threshold_pairs)
plot_recall(metrics)
metrics.groupby('source')['recall_0.05_0.95'].mean()